In [17]:
from KaggleLoader import KaggleDataLoader
import torchmetrics
from torchvision import  transforms
import torch
from torchmetrics import MetricCollection, Accuracy,Precision,ConfusionMatrix,CohenKappa,Specificity,Recall
from torch.utils.data import Dataset,DataLoader
from tqdm.notebook import tqdm
from timm.models import create_model

In [22]:
t = []
size = int((256 / 224) * 224)
t.append(
    transforms.Resize(size, interpolation=transforms.InterpolationMode.BICUBIC),  # to maintain same ratio w.r.t. 224 images
)
t.append(transforms.CenterCrop(224))
t.append(transforms.ToTensor())
t.append(transforms.Normalize((81.69452533289052,57.23439034767294,41.10348823578685), (77.14657522678525,55.77959643810525,44.47320493966664))) #Mean,STD
transform = transforms.Compose(t)


data = KaggleDataLoader("/mnt/f/DataSet/DRDetectionKaggle/retinopathy_solution.csv","/mnt/f/DataSet/DRDetectionKaggleReduce/test",transform,"private")

dataloader = DataLoader(data, batch_size=50,num_workers=8, shuffle=False)


device = torch.device("cuda")
checkpoint = torch.load("/mnt/c/Users/Romain/Downloads/Run4CLAHE/checkpoint_25.pth")

model = create_model(
        'convit_small',
        pretrained=False,
        img_size=224,
        num_classes=5
    )

model.load_state_dict(checkpoint['model'])
#print(model)
# Make sure to call input = input.to(device) on any input tensors that you feed to the model
model.eval()
model.to(device)

metrics = MetricCollection([
    Accuracy(num_classes=5),
    Precision(num_classes=5),
    ConfusionMatrix(num_classes=5),
    CohenKappa(num_classes=5),
    Specificity(num_classes=5),
    Recall(num_classes=5)
])
with torch.no_grad():
    for samples, targets in tqdm(dataloader):
        samples = samples.to(device)
        preds = model(samples)
        preds=preds.to("cpu")
        m = torch.nn.Softmax(dim=1)
        preds = m(preds)

        #metrics
        metrics(preds,targets)
print(metrics)


  0%|          | 0/1072 [00:00<?, ?it/s]

tensor([[0.6293, 0.0920, 0.1721, 0.0621, 0.0444],
        [0.6290, 0.0918, 0.1726, 0.0621, 0.0445],
        [0.6294, 0.0920, 0.1722, 0.0621, 0.0444],
        [0.6290, 0.0919, 0.1725, 0.0622, 0.0444],
        [0.6293, 0.0919, 0.1723, 0.0621, 0.0444],
        [0.6292, 0.0920, 0.1723, 0.0621, 0.0445],
        [0.6294, 0.0919, 0.1722, 0.0621, 0.0443],
        [0.6292, 0.0919, 0.1724, 0.0621, 0.0444],
        [0.6289, 0.0919, 0.1725, 0.0622, 0.0444],
        [0.6290, 0.0919, 0.1724, 0.0622, 0.0445],
        [0.6287, 0.0919, 0.1726, 0.0623, 0.0445],
        [0.6289, 0.0919, 0.1726, 0.0622, 0.0444],
        [0.6287, 0.0919, 0.1727, 0.0623, 0.0445],
        [0.6287, 0.0920, 0.1726, 0.0622, 0.0444],
        [0.6285, 0.0919, 0.1728, 0.0623, 0.0445],
        [0.6279, 0.0918, 0.1731, 0.0624, 0.0448],
        [0.6286, 0.0918, 0.1728, 0.0623, 0.0445],
        [0.6283, 0.0919, 0.1730, 0.0623, 0.0445],
        [0.6295, 0.0920, 0.1720, 0.0621, 0.0443],
        [0.6293, 0.0920, 0.1723, 0.0621, 0.0444],


KeyboardInterrupt: 

In [4]:
metrictest = metrics
print(metrictest.compute())

{'Accuracy': tensor(0.2000), 'Precision': tensor(0.1476), 'ConfusionMatrix': tensor([[39533,     0,     0,     0,     0],
        [ 3762,     0,     0,     0,     0],
        [ 7861,     0,     0,     0,     0],
        [ 1214,     0,     0,     0,     0],
        [ 1206,     0,     0,     0,     0]]), 'CohenKappa': tensor(0.), 'Specificity': tensor(0.8000), 'Recall': tensor(0.2000)}


In [5]:
import pickle
with open('saved_dictionaryMetrics.pkl', 'wb') as f:
    pickle.dump(metrics, f)
        
"""with open('saved_dictionary.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)"""

"with open('saved_dictionary.pkl', 'rb') as f:\n    loaded_dict = pickle.load(f)"